In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from glob import glob
from collections import defaultdict
from PIL import Image
import numpy as np
import cv2

# 1. Images Metadata

In [ ]:
# 학습 데이터의 경로와 정보를 가진 파일의 경로를 설정
traindata_dir = "./data/train"
traindata_info_file = "./data/train.csv"

# 테스트 데이터의 경로와 정보를 가진 파일의 경로를 설정
testdata_dir = "./data/test"
testdata_info_file = "./data/test.csv"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 학습 데이터의 class, image path, target에 대한 정보가 들어있는 csv파일을 읽기
train_data = pd.read_csv(traindata_info_file)

# 테스트 데이터
test_data = pd.read_csv(testdata_info_file)

In [ ]:
# 학습 데이터의 정보를 출력
train_info = train_data.info()
train_head = train_data.head()

train_info, train_head

주어진 학습 데이터셋은 15021개의 항목과 3개의 컬럼으로 구성되어 있다.
1. class_name: 클래스 이름, string
2. image_path: 이미지 파일의 경로, string
3. target: 클래스를 의미하는 숫자아이디, integer

In [ ]:
# 테스트 데이터의 정보를 출력.
test_info = test_data.info()
test_head = test_data.head()

test_info, test_head

주어진 테스트 데이터셋은 10014개의 항목과 1개의 컬럼으로 구성되어 있다.
1. image_path: 이미지 파일의 경로, string

In [ ]:
# 데이터의 기본적인 통계 정보를 출력
data_description = train_data.describe(include='all')

# class_name의 unique한 값의 개수를 출력
unique_classes = train_data['class_name'].nunique()

# target의 unique한 값의 개수를 출력
unique_targets = train_data['target'].nunique()

data_description, unique_classes, unique_targets

기본 통계
- 데이터셋에는 15,021개의 항목이 있음
- 500개의 고유한 클래스 이름과 15021개의 고유한 이미지 경로가 존재
- Target값은 0에서 499까지 500개의 값을 가지고 있음

In [ ]:
plt.figure(figsize=(16, 10))

# class_name별로 샘플의 개수를 출력
sns.countplot(y=train_data['class_name'], order=train_data['class_name'].value_counts().index, palette='viridis')
plt.title('Distribution of Samples per Class')
plt.xlabel('Number of Samples')
plt.ylabel('Class Name')

# target 값의 분포를 출력
plt.figure(figsize=(16, 10))
sns.histplot(train_data['target'], bins=500, kde=False, palette='viridis')
plt.title('Distribution of Target Values')
plt.xlabel('Target Value')
plt.ylabel('Number of Samples')

plt.show()

Class name 분포
- 대부분 29에서 31 사이의 값을 가지고 있음

Target 분포
- 대부분 29에서 31 사이의 값을 가지고 있음

데이터 셋 전반적으로 Class name, Target에 대해서 29~31개의 값을 가지고 있음.

# 2. Images - Exploration and processing

In [ ]:
# glob을 이용하여 이미지 파일의 경로를 읽어옴
train_images = glob(traindata_dir + "/*/*")
test_images = glob(testdata_dir + "/*")
print(f"Number of train images: {len(train_images)}")
print(f"Number of test images: {len(test_images)}")

## 2.1 Getting Image's statistics

In [ ]:
image_prop = defaultdict(list)

for i, path in enumerate(train_images):
    with Image.open(path) as img:
        image_prop['height'].append(img.height)
        image_prop['width'].append(img.width)
        image_prop['img_aspect_ratio'] = img.width / img.height
        image_prop['mode'].append(img.mode)
        image_prop['format'].append(img.format)
        image_prop['size'].append(round(os.path.getsize(path) / 1e6, 2))
    image_prop['path'].append(path)
    image_prop['image_path'].append(path.split('/')[-2] + "/" + path.split('/')[-1])

image_data = pd.DataFrame(image_prop)

image_data = image_data.merge(train_data, on='image_path')
#image_data.sort_values(by='target', inplace=True)


In [ ]:
# 이미지의 특징을 추출하는 함수
def extract_image_features(image_path):
    """
    Extracts features from an image.
    Args:
        image_path (str): Path to the image file.
    Returns:
        width (int): Width of the image.
        height (int): Height of the image.
        mode (str): Mode of the image.
        format (str): Format of the image.
        size (int): Size of the image.
        mean_red (float): Mean of red channel.
        mean_green (float): Mean of green channel.
        mean_blue (float): Mean of blue channel.
    """
    try:
        with Image.open(image_path) as img:
            img = img.convert('RGB')
            width, height = img.size
            img_array = np.array(img)
            mean_red = np.mean(img_array[:, :, 0])
            mean_green = np.mean(img_array[:, :, 1])
            mean_blue = np.mean(img_array[:, :, 2])
            format = image_path.split('.')[-1].upper()
            return width, height, img.mode, format, os.path.getsize(image_path), mean_red, mean_green, mean_blue
    except Exception as e:
        return None, None, None, None, None, None, None, None

image_prop = defaultdict(list)

for i, path in enumerate(train_images):
    width, height, mode, format, size, mean_red, mean_green, mean_blue = extract_image_features(path)
    image_prop['height'].append(height)
    image_prop['width'].append(width)
    image_prop['mode'].append(mode)
    image_prop['format'].append(format)
    image_prop['size'].append(round(size / 1e6, 2) if size else None)
    image_prop['mean_red'].append(mean_red)
    image_prop['mean_green'].append(mean_green)
    image_prop['mean_blue'].append(mean_blue)
    image_prop['path'].append(path)
    image_prop['image_path'].append(path.split('/')[-2] + "/" + path.split('/')[-1])

image_data = pd.DataFrame(image_prop)
image_data['img_aspect_ratio'] = image_data['width'] / image_data['height']

image_data = image_data.merge(train_data, on='image_path')
image_data.sort_values(by='target', inplace=True)


### 2.1.1 이미지 파일크기 분석

In [ ]:
plt.figure(figsize=(16, 10))

# 이미지 파일의 크기 분포를 출력
sns.histplot(image_data['size'], bins=30, kde=True, color='green')
plt.title('Distribution of Image File Sizes')
plt.xlabel('Size (MB)')
plt.ylabel('Frequency')

plt.show()

이미지 파일 크기 분포
- 대부분 이미지 파일 크기는 0.05 ~ 0.1 MB 사이에 분포

### 2.1.2 이미지 파일크기 분석

In [ ]:
plt.figure(figsize=(16, 10))
# Image height의 분포를 출력
plt.subplot(1, 2, 1)
sns.histplot(image_data['height'], bins=30, kde=True, color='skyblue')
plt.title('Distribution of Image Height')
plt.xlabel('Height')
plt.ylabel('Frequency')

# Image width의 분포를 출력
plt.subplot(1, 2, 2)
sns.histplot(image_data['width'], bins=30, kde=True, color='orange')
plt.title('Distribution of Image Width')
plt.xlabel('Width')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

이미지의 높이와 너비 분포
- 이미지 높이: 대부분 이미지 높이는 400에서 800 픽셀 사이에 분포
- 이미지 너비: 대부분 이미지 너비는 400에서 800 픽셀 사이에 분포

In [ ]:
plt.figure(figsize=(10, 6))

# 이미지의 가로 세로 비율을 출력
sns.histplot(image_data['img_aspect_ratio'], bins=30, kde=True, color='purple')
plt.title('Distribution of Image Aspect Ratios')
plt.xlabel('Aspect Ratio')
plt.ylabel('Frequency')

plt.show()


가로세로 비율 분포
- 대부분의 이미지 가로세로 비율을 0.8에서 1.2사이에 분포

In [ ]:
# 10개의 샘플 선택
sample_classes = image_data['class_name'].unique()[:10]

# 각 클래스별로 RGB값의 평균을 계산
mean_rgb_per_class = image_data[image_data['class_name'].isin(sample_classes)].groupby('class_name')[['mean_red', 'mean_green', 'mean_blue']].mean()

# 클래스별로 RGB값의 평균을 출력
mean_rgb_per_class.plot(kind='bar', figsize=(16, 10), color=['red', 'green', 'blue'])
plt.title('Mean RGB Values per Class')
plt.xlabel('Class Name')
plt.ylabel('Mean RGB Value')
plt.xticks(rotation=45)
plt.legend(['Mean Red', 'Mean Green', 'Mean Blue'])
plt.show()


In [ ]:
# RGB값의 평균 분포를 출력
plt.figure(figsize=(14, 6))

# mean_red 값의 분포를 출력
plt.subplot(1, 3, 1)
sns.histplot(image_data['mean_red'], bins=30, kde=True, color='red')
plt.title('Distribution of Mean Red Values')
plt.xlabel('Mean Red Value')
plt.ylabel('Frequency')

# mean_green 값의 분포를 출력
plt.subplot(1, 3, 2)
sns.histplot(image_data['mean_green'], bins=30, kde=True, color='green')
plt.title('Distribution of Mean Green Values')
plt.xlabel('Mean Green Value')
plt.ylabel('Frequency')

# mean_blue 값의 분포를 출력
plt.subplot(1, 3, 3)
sns.histplot(image_data['mean_blue'], bins=30, kde=True, color='blue')
plt.title('Distribution of Mean Blue Values')
plt.xlabel('Mean Blue Value')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()


색상 분포 분석
- 대부분 RGB, Green, Blue값이 200~250사이에 분포
- 전박적으로 이미지의 밝기와 채도가 높은 편

In [ ]:
# 각 클래스별로 이미지의 평균 높이와 너비를 계산
class_size_stats = image_data.groupby('class_name')[['height', 'width']].mean().reset_index()

# 각 클래스별로 이미지의 평균 높이와 너비를 출력
plt.figure(figsize=(16, 10))

# 클래스별 이미지의 평균 높이를 출력
plt.subplot(1, 2, 1)
sns.barplot(x='height', y='class_name', data=class_size_stats.sort_values(by='height', ascending=False), palette='viridis')
plt.title('Mean Image Height by Class')
plt.xlabel('Mean Height')
plt.ylabel('Class Name')

# 클래스별 이미지의 평균 너비를 출력
plt.subplot(1, 2, 2)
sns.barplot(x='width', y='class_name', data=class_size_stats.sort_values(by='width', ascending=False), palette='viridis')
plt.title('Mean Image Width by Class')
plt.xlabel('Mean Width')
plt.ylabel('Class Name')

plt.tight_layout()
plt.show()

클래스별 이미지 크기 분포
- 대부분 클래스는 300에서 800 픽셀 사이의 평균 높이를 가지고 있음
- 대부분 클래스느 300에서 800 픽셀 사이의 평균 너비를 가지고 있음

In [ ]:
plt.figure(figsize=(16, 10))

# mean_red 값의 분포를 출력
plt.subplot(1, 3, 1)
sns.boxplot(x='target', y='mean_red', data=image_data, palette='Reds')
plt.title('Mean Red Value Distribution by Target')
plt.xlabel('Target')
plt.ylabel('Mean Red Value')
plt.xticks(rotation=90)

# mean_green 값의 분포를 출력
plt.subplot(1, 3, 2)
sns.boxplot(x='target', y='mean_green', data=image_data, palette='Greens')
plt.title('Mean Green Value Distribution by Target')
plt.xlabel('Target')
plt.ylabel('Mean Green Value')
plt.xticks(rotation=90)

# mean_blue 값의 분포를 출력
plt.subplot(1, 3, 3)
sns.boxplot(x='target', y='mean_blue', data=image_data, palette='Blues')
plt.title('Mean Blue Value Distribution by Target')
plt.xlabel('Target')
plt.ylabel('Mean Blue Value')
plt.xticks(rotation=90)

plt.tight_layout()
plt.show()


클래스별 이미지 색상의 분포
- 대부분 Red, Green, Blue 값은 200에서 250 사이에 분포

## 2.2 Displaying images

In [ ]:
# 같은 target을 가진 이미지 전체 출력
def display_images(data, target):
    len_data = len(data[data['target'] == target])
    fig, axs = plt.subplots((len_data // 5)+1, 5, figsize=(16, 10))
    images = data[data['target'] == target]['path'].values
    for i, path in enumerate(images):
        img = Image.open(path)
        ax = axs[i // 5, i % 5]  # Use double indexing for 2D subplots
        ax.imshow(img)
        ax.axis('off')
    plt.show()

# target이 0인 이미지 출력
display_images(image_data, target=1)

### 2.2.1 Displying random images using PIL

In [ ]:
# 이미지를 랜덤으로 5개 출력
plt.style.use('default')
fig, axex = plt.subplots(2, 5, figsize=(16, 10))
for ax in axex.reshape(-1):
    img_path = np.random.choice(train_images)
    img = Image.open(img_path)
    ax.imshow(img)
    ax.set_title(f"Image name: {img_path.split('/')[-1]}")
plt.show()

In [ ]:
# 가장 큰 이미지를 출력
biggest_img_path = image_data.iloc[image_data['size'].idxmax(),:]['path']

img = Image.open(biggest_img_path)
plt.title(f"Biggest image: {biggest_img_path.split('/')[-1]}")
plt.imshow(img)
del img

In [ ]:
# 4개의 작은 이미지를 출력
smallest_img_paths = image_data.nsmallest(4, 'size')['path']
smallest_img_paths.values

In [ ]:
import matplotlib.image as mpimg

# 4개의 작은 이미지를 출력
fig, axes = plt.subplots(1, 4, figsize=(16, 10))
for i, ax in enumerate(axes.reshape(-1)):
    img = mpimg.imread(smallest_img_paths.values[i])
    ax.title.set_text(f"Smallest image: {smallest_img_paths.values[i].split('/')[-1]}")
    ax.imshow(img)
plt.show()

In [ ]:
# 이미지의 종횡비가 가장 큰 이미지를 출력
biggest_aspect_ratio_img_path = image_data.iloc[image_data['img_aspect_ratio'].idxmax(),:]['path']

fig, ax = plt.subplots(1, 1, figsize=(16, 10))
img = Image.open(biggest_aspect_ratio_img_path)
plt.title(f"Biggest aspect ratio image: {biggest_aspect_ratio_img_path.split('/')[-1]}")
plt.imshow(img)

### 2.2.2 Diplaying, resizing and manipulation using CV2

In [ ]:
img_path = train_images[10]
selected_img = image_data[image_data['path'] == img_path]
selected_img

In [ ]:
img = mpimg.imread(img_path)
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
img = cv2.imread(img_path, cv2.IMREAD_COLOR)
resized_img = cv2.resize(img, (0,0), fx=0.5, fy=0.5)

fig, ax = plt.subplots(1, 2, figsize=(16, 10))
ax[0].imshow(img)
ax[0].set_title("Original image")
ax[1].imshow(resized_img)
ax[1].set_title("Resized image")
plt.show()

# Failure Analysis

In [ ]:
class DataProcessor:
    def __init__(self, train_df, output_df, target_folder='/content/data/'):
        self.train_df = train_df
        self.output_df = output_df
        self.target_folder = target_folder

    # 클래스 내부에서만 처리

    # 전처리 output_df 전처리
    def _preprocess_df(self):
        # output_df ID열 삭제
        self.output_df.drop('ID', axis=1, inplace=True, errors='ignore')

        # output_df 컬럼명 변경
        self.output_df = self.output_df.rename(columns={'image_path' : 'test_image_path', 'target' : 'predicted_target'})

        return self.output_df

    # 데이터 폴더에 ._ 되어있는 파일 삭제 함수
    def _delete_useless_files(self):
        # os.walk()를 사용하여 폴더와 하위 폴더를 재귀적으로 탐색
        for root, _, files in os.walk(self.target_folder):
            for filename in files:
                # 파일명이 '._'로 시작하는지 확인
                if filename.startswith('._'):
                    file_path = os.path.join(root, filename)
                    # 해당 파일 삭제
                    os.remove(file_path)

    def visualize_predicted(self, idx, train_dir, test_dir):
        self._preprocess_df()
        self._delete_useless_files()

        # idx에 해당하는 테스트 이미지 경로
        test_path = os.path.join(test_dir, self.output_df['test_image_path'].iloc[idx].upper())
        # idx에 해당하는 훈련 이미지 폴더 경로
        train_path = os.path.join(train_dir, self.output_df['class_name'].iloc[idx])

        # 예측한 데이터의 클래스 폴더명
        class_name = self.output_df['class_name'].iloc[idx]

        # test_path에 해당하는 테스트 이미지
        test_image = Image.open(test_path).convert('RGB')

        # 테스트 이미지 시각화
        plt.figure(figsize=(3, 3))
        plt.imshow(test_image)
        plt.title('Test Image')
        plt.axis('off')
        plt.show(block=False)

        # 테스트 이미지를 보고 예측한 target 폴더 순회하며 이미지 시각화
        for dirpath, _, files in os.walk(train_path):

            # 폴더 안 이미지 개수, num_diplay 중 작은 값
            num_images = len(files)

            # 폴더 이미지 기준으로 cols, rows 지정
            cols = math.ceil(math.sqrt(num_images))
            rows = math.ceil(num_images / cols)

            # 서브 플롯 생성
            print(idx)
            fig, axes = plt.subplots(rows, cols, figsize=(cols * 8, rows * 4))

            print()
            print(f'Target Label : {label}')

            if rows == 1 or cols == 1:
                axes = axes.flatten()

            # 폴더를 순회하며 이미지 시각화
            for i, file_name in enumerate(files):
                # JPEG 확장자만 처리
                if file_name.lower().endswith('.jpeg'):
                    # 각 이미지 경로
                    target_dir = os.path.join(dirpath, file_name)
                    image = Image.open(target_dir).resize((128, 128)).convert('RGB')
                    # 서브 플롯 계산
                    if isinstance(axes, np.ndarray):
                        if len(axes.shape) == 1:
                            ax = axes[i]
                        else:
                            ax = axes[i // cols, i % cols]
                    else:
                        ax = axes

                ax.imshow(image)
                ax.axis('off')

        plt.tight_layout()
        # 빈 서브플롯 남았을 경우 삭제
        for j in range(num_images, rows * cols):
            fig.delaxes(axes.flatten()[j])

        plt.show()
        print()
        print('-----------------------------------------------------------------------------------------------------')
        print()

    def get_predicted_indices_by_class(self, class_name):
        # 전처리 수행
        self._preprocess_df()

        # 클래스 이름을 대문자로 변환하여 일관성 유지
        class_name_upper = class_name.upper()

        # `output_df`에서 해당 클래스를 예측한 인덱스들을 찾은 후 리스트로 반환
        indices = self.output_df[self.output_df['class_name'].str.upper() == class_name_upper].index.tolist()

        # 인덱스를 반환
        return indices

    def visualize_images_by_class(self, class_name, train_dir, test_dir, max_images=None):
        """
        :param class_name: 시각화할 클래스 이름
        :param train_dir: 훈련 데이터 디렉토리
        :param test_dir: 테스트 데이터 디렉토리
        :param max_images: 시각화할 최대 이미지 수, 기본값은 None (모든 이미지를 시각화)
        :return: None
        """
        # 해당 class_name으로 예측된 인덱스 가져오기
        indices = self.get_predicted_indices_by_class(class_name)

        # 최대 이미지 수 제한 (max_images가 설정되어 있을 때만)
        if max_images is not None:
            indices = indices[:max_images]

        print(f'Class Name: {class_name}, Visualizing {len(indices)} images.')

        # 각 인덱스의 이미지를 시각화
        num_images = len(indices)
        if num_images == 0:
            print(f"No images found for class '{class_name}'")
            return

        # 지정할 행과 열의 개수 (이미지를 그릴 수 있는 그리드 크기)
        cols = math.ceil(math.sqrt(num_images))
        rows = math.ceil(num_images / cols)

        fig, axes = plt.subplots(rows, cols, figsize=(cols * 8, rows * 4))

        if rows == 1 or cols == 1:
            axes = axes.flatten()

        # 인덱스를 순회하며 테스트 이미지들을 시각화
        for i, idx in enumerate(indices):
            # idx에 해당하는 테스트 이미지 경로를 계산
            test_path = os.path.join(test_dir, self.output_df['test_image_path'].iloc[idx].upper())
            test_image = Image.open(test_path).convert('RGB')
            test_image = test_image.resize((128, 128))

            # 서브 플롯에 이미지 추가
            if isinstance(axes, np.ndarray):
                if len(axes.shape) == 1:
                    ax = axes[i]
                else:
                    ax = axes[i // cols, i % cols]
            else:
                ax = axes

            ax.imshow(test_image)
            ax.set_title(f"Idx: {idx}")
            ax.axis('off')

        # 빈 서브플롯 삭제 (필요한 경우)
        for j in range(num_images, rows * cols):
            fig.delaxes(axes.flatten()[j])

        plt.tight_layout()
        plt.show()

In [ ]:
# 경로 정의

# Aistage에서 다운받은 output.csv 경로
output_path = '/content/output.csv'
output_df = pd.read_csv(output_path)

# train 데이터 셋
train_df = pd.read_csv('/content/data/train.csv')

train_dir = '/content/data/train'
test_dir = '/content/data/test'

In [ ]:
# 클래스 선언
processor = DataProcessor(train_df=train_df, output_df=output_df, class_dict=class_dict)

In [ ]:
processor.visualize_predicted(1, train_dir, test_dir)

In [ ]:
processor.visualize_images_by_class('n03598930', train_dir, test_dir)  # 원하는 클래스를 넣어줍니다.

# 중복 이미지 탐지

- 서로 다른 클래스 내에 중복 이미지를 탐지하고 딕셔너리 형태로 저장하는 Class

In [ ]:
def images_are_exactly_equal(image1_path, image2_path):
    """두 이미지의 픽셀값을 직접 비교하는 함수 (손상된 파일 및 숨김 파일 예외 처리)"""
    try:
        with Image.open(image1_path) as img1, Image.open(image2_path) as img2:
            # 이미지 크기 비교
            if img1.size != img2.size:
                return False

            # 이미지를 NumPy 배열로 변환하여 픽셀값 비교
            img1_array = np.array(img1)
            img2_array = np.array(img2)

            return np.array_equal(img1_array, img2_array)
    except Exception as e:
        print(f"이미지 비교 오류: {image1_path}, {image2_path}, 오류: {e}")
        return False

def find_and_print_duplicates_across_folders(folder_path):
    """각 폴더 내 이미지들을 다른 폴더 내 이미지와 비교하여 중복된 이미지 쌍을 출력하는 함수"""
    folder_images = {}  # 각 폴더의 이미지 파일을 저장할 딕셔너리

    # 모든 폴더와 이미지 파일을 수집 (각 폴더 별로)
    for root, dirs, files in os.walk(folder_path):
        image_files = []

        # 각 폴더 내 이미지 파일 수집
        for file in files:
            if not file.startswith('._') and file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif')):
                file_path = os.path.join(root, file)
                image_files.append(file_path)

        if image_files:
            folder_images[root] = image_files

    # 각 폴더의 이미지 파일을 다른 폴더와 비교
    folder_keys = list(folder_images.keys())

    for i in range(len(folder_keys)):
        for j in range(i + 1, len(folder_keys)):
            folder1 = folder_keys[i]
            folder2 = folder_keys[j]

            images_folder1 = folder_images[folder1]
            images_folder2 = folder_images[folder2]

            # 두 폴더의 이미지 파일 비교
            for image1 in images_folder1:
                for image2 in images_folder2:
                    if images_are_exactly_equal(image1, image2):
                        print(f"중복 이미지 발견: {image2} (다른 폴더), 원본: {image1}")

In [ ]:
folder_path = "/content/data/train"
find_and_print_duplicates_across_folders(folder_path)